In [1]:
import pandas as pd, numpy as np, warnings, os
#import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import RFE
import pickle

In [2]:
pwd

'/Users/adityabhatkal/Desktop/Practice 32'

In [3]:
warnings.filterwarnings('ignore')

In [4]:
pwd

'/Users/adityabhatkal/Desktop/Practice 32'

In [5]:
#os.chdir('C:\\Users\\ABC\\6. Datasets')

In [6]:
df = pd.read_csv('superstore_data_Classifier - 82% accuracy.csv')
df.columns

Index(['Id', 'Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome',
       'Teenhome', 'Dt_Customer', 'Recency', 'MntWines', 'MntFruits',
       'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
       'Complain', 'Response'],
      dtype='object')

In [7]:
df

,Id,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Complain,Response
0,1826,1970,Graduation,Divorced,84835.0,0,0,6/16/2014,0,189,...,111,189,218,1,4,4,6,1,0,1
1,1,1961,Graduation,Single,57091.0,0,0,6/15/2014,0,464,...,7,0,37,1,7,3,7,5,0,1
2,10476,1958,Graduation,Married,67267.0,0,1,5/13/2014,0,134,...,15,2,30,1,3,2,5,2,0,0
3,1386,1967,Graduation,Together,32474.0,1,1,11/5/2014,0,10,...,0,0,0,1,1,0,2,7,0,0
4,5371,1989,Graduation,Single,21474.0,1,0,8/4/2014,0,6,...,11,0,34,2,3,1,2,7,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,10142,1976,PhD,Divorced,66476.0,0,1,7/3/2013,99,372,...,47,48,78,2,5,2,11,4,0,0
2236,5263,1977,2n Cycle,Married,31056.0,1,0,1/22/2013,99,5,...,3,8,16,1,1,0,3,8,0,0
2237,22,1976,Graduation,Divorced,46310.0,1,0,3/12/2012,99,185,...,15,5,14,2,6,1,5,8,0,0
2238,528,1978,Graduation,Married,65819.0,0,0,11/29/2012,99,267,...,149,165,63,1,5,4,10,3,0,0


In [8]:
df.columns
df.drop(['Id', 'Year_Birth', 'Dt_Customer'], inplace=True, axis = 1)

In [9]:
df['Education'] = df['Education'].map({
    'Basic':0,
    '2n Cycle':1,
    'Graduation':2,
    'Master':3,
    'PhD':4   
})

In [10]:
#Checking the below proportions of 1vs0 for double vs single,  singles opt in for schemes a lot more.
#SO giving them a 2
df['Marital_Status'].value_counts()

df['Marital_Status'] = df['Marital_Status'].map({
    'Married':1,
    'Together':1,
    'Single':2,
    'Widow':2,
    'Divorced':2,
    'YOLO':2,
    'Alone':2,
    'Absurd':2
})

In [11]:
listy = []
for i in df['Marital_Status'].values:
    if i not in listy:
        listy.append(i)
listy

[np.int64(2), np.int64(1)]

In [12]:
df[df['Marital_Status']=='Divorced']['Response'].value_counts()


Series([], Name: count, dtype: int64)

In [13]:
df[df['Marital_Status']=='Together']['Response'].value_counts()


Series([], Name: count, dtype: int64)

In [14]:
df[df['Marital_Status']=='Single']['Response'].value_counts()


Series([], Name: count, dtype: int64)

In [15]:
df[df['Marital_Status']=='Widow']['Response'].value_counts()


Series([], Name: count, dtype: int64)

In [16]:
df[df['Marital_Status']=='Married']['Response'].value_counts()


Series([], Name: count, dtype: int64)

In [17]:
df

,Education,Marital_Status,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Complain,Response
0,2,2,84835.0,0,0,0,189,104,379,111,189,218,1,4,4,6,1,0,1
1,2,2,57091.0,0,0,0,464,5,64,7,0,37,1,7,3,7,5,0,1
2,2,1,67267.0,0,1,0,134,11,59,15,2,30,1,3,2,5,2,0,0
3,2,1,32474.0,1,1,0,10,0,1,0,0,0,1,1,0,2,7,0,0
4,2,2,21474.0,1,0,0,6,16,24,11,0,34,2,3,1,2,7,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,4,2,66476.0,0,1,99,372,18,126,47,48,78,2,5,2,11,4,0,0
2236,1,1,31056.0,1,0,99,5,10,13,3,8,16,1,1,0,3,8,0,0
2237,2,2,46310.0,1,0,99,185,2,88,15,5,14,2,6,1,5,8,0,0
2238,2,1,65819.0,0,0,99,267,38,701,149,165,63,1,5,4,10,3,0,0


In [18]:
df['Income'].fillna(52247.251354, inplace=True)

In [19]:
#People who bought into the product v/s thse who didnt complain!
df[df['Response']==1]['Complain'].value_counts()

Complain
0    331
1      3
Name: count, dtype: int64

In [20]:
df.head(2)

,Education,Marital_Status,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Complain,Response
0,2,2,84835.0,0,0,0,189,104,379,111,189,218,1,4,4,6,1,0,1
1,2,2,57091.0,0,0,0,464,5,64,7,0,37,1,7,3,7,5,0,1


In [21]:
x = df.iloc[:,[0, 1, 2, 3, 4,7,  13, 14, 15, 16, 17]]
y = df.iloc[:,-1]

In [22]:
df.isnull().sum()

Education              0
Marital_Status         0
Income                 0
Kidhome                0
Teenhome               0
Recency                0
MntWines               0
MntFruits              0
MntMeatProducts        0
MntFishProducts        0
MntSweetProducts       0
MntGoldProds           0
NumDealsPurchases      0
NumWebPurchases        0
NumCatalogPurchases    0
NumStorePurchases      0
NumWebVisitsMonth      0
Complain               0
Response               0
dtype: int64

In [23]:
for i in df.columns:
    df[i] = df[i].astype(int)

In [24]:
for i in df.iloc[1,:]:
    print(type(i))

<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>


In [25]:
#Logistic Regresion for 

x_train_lr, x_test_lr, y_train_lr, y_test_lr = train_test_split(x,y,test_size=0.2, random_state=i)
lg = LogisticRegression(max_iter=1000)
model_lr = lg.fit(x_train_lr, y_train_lr)
y_predict_lr = model_lr.predict(x_test_lr)
acc_lr = accuracy_score(y_predict_lr, y_test_lr)
cv = cross_val_score(model_lr, x, y, cv=5, scoring='accuracy')
if acc_lr>0.59:
    print(i, acc_lr)
    print(cv) 

1 0.8325892857142857
[0.83705357 0.85491071 0.85714286 0.84375    0.84151786]


In [26]:
#FEatures to be selected
rfe = RFE(model_lr, n_features_to_select=9)
fitter = rfe.fit(x,y) 
feat = []
[feat.append(i) for i,j in enumerate(fitter.support_) if j == True]
feat

[0, 1, 3, 4, 6, 7, 8, 9, 10]

In [27]:
x_train_svc, x_test_svc, y_train_svc, y_test_svc = train_test_split(x,y,test_size = 0.2,
                                                                    random_state = 5)
svc = SVC(C=0.1, probability=True)
model_svc = svc.fit(x_train_svc, y_train_svc)
y_predict_svc = model_svc.predict(x_test_svc)
acc_svc = accuracy_score(y_test_svc, y_predict_svc)
cv = cross_val_score(svc, x,y,scoring='accuracy', cv=StratifiedKFold(n_splits=5))
if acc_svc > 0.8:
    print(cv, acc_svc)

[0.85044643 0.85044643 0.85044643 0.85044643 0.85267857] 0.8370535714285714


In [28]:
[i for i in x.iloc[4, :]]

[2.0, 2.0, 21474.0, 1.0, 0.0, 16.0, 3.0, 1.0, 2.0, 7.0, 0.0]

In [29]:
print(model_svc.predict_proba([[2.0, 2.0, 21474.0, 1.0, 0.0, 16.0, 3.0, 1.0, 2.0, 7.0, 0.0]])[0][1])
print(model_svc.predict([[2.0, 2.0, 21474.0, 1.0, 0.0, 16.0, 3.0, 1.0, 2.0, 7.0, 0.0]])[0])

0.10017437733122643
0


In [30]:
with open('customerbuyin.sav', 'wb') as f1:
    pickle.dump(model_svc, f1)

In [31]:
with open('customerbuyin.sav', 'rb') as f2:
    cby_loaded_model = pickle.load(f2)

In [32]:
cby_loaded_model.predict_proba([[2.0, 2.0, 21474.0, 1.0, 0.0, 16.0, 3.0, 1.0, 2.0, 7.0, 0.0]])[0][1]

np.float64(0.10017437733122643)

In [33]:
x.columns

Index(['Education', 'Marital_Status', 'Income', 'Kidhome', 'Teenhome',
       'MntFruits', 'NumWebPurchases', 'NumCatalogPurchases',
       'NumStorePurchases', 'NumWebVisitsMonth', 'Complain'],
      dtype='object')